<a href="https://colab.research.google.com/github/seodalzzz/ESAA/blob/Exercises/CLASSIFICATION_exercise.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. 결정 트리 훈련

1-a . make_moon ( n_sample=1000, noise=0.4)를 사용해 데이터셋을 생성합니다.

In [49]:
from sklearn.datasets import make_moons

X,y = make_moons(n_samples=1000, noise=0.4, random_state=42)

1-b. 이를 train_test_split( )을 사용해 훈련 세트와 테스트 세트로 나눕니다.(random_state=42)


In [50]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42)

1-c. DecisionTreeClassifier의 최적의 매개변수를 찾기 위해 교차 검증과 함께 그리드 탐색을 수행합니다. (GridSearchCV) 

힌트 : 여러가지 max_leaf_nodes 값을 시도해보세요! (교재 113, 207p 참고)

In [51]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV

# 예시 파라미터: params = {'max_leaf_nodes' : list(range(2,100)),'min_samples_split' : [2,3,4]}

tree_clf=DecisionTreeClassifier(random_state=42)

params={'max_leaf_nodes':list(range(2,100)),'min_samples_split':[2,3,4]}

grid_search_cv=GridSearchCV(tree_clf,param_grid=params,cv=3,n_jobs=-1,verbose=1)

grid_search_cv.fit(X_train,y_train)

Fitting 3 folds for each of 294 candidates, totalling 882 fits


GridSearchCV(cv=3, estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'max_leaf_nodes': [2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12,
                                            13, 14, 15, 16, 17, 18, 19, 20, 21,
                                            22, 23, 24, 25, 26, 27, 28, 29, 30,
                                            31, ...],
                         'min_samples_split': [2, 3, 4]},
             verbose=1)

In [52]:
grid_search_cv.best_estimator_

DecisionTreeClassifier(max_leaf_nodes=4, random_state=42)

1-d. 찾은 매개변수를 사용해 전체 훈련 세트에 대해 모델을 훈련시키고 테스트 세트에서 성능(정확도)을 측정합니다. (대략 85~87% 정도 나옵니다.)

In [53]:
# GridSearchCV는 최적의 모델로 다시 훈련시키기 때문에 별도의 훈련이 필요없습니다.
from sklearn.metrics import accuracy_score
y_pred=grid_search_cv.best_estimator_.predict(X_test)
accuracy_score(y_test,y_pred)

0.855

# 2. 랜덤포레스트

In [8]:
# 데이터 다운로드 링크로 데이터를 코랩에 불러옵니다.

!wget 'https://bit.ly/3i4n1QB'

import zipfile
with zipfile.ZipFile('3i4n1QB', 'r') as existing_zip:
    existing_zip.extractall('data')

--2022-05-09 08:28:09--  https://bit.ly/3i4n1QB
Resolving bit.ly (bit.ly)... 67.199.248.11, 67.199.248.10
Connecting to bit.ly (bit.ly)|67.199.248.11|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://drive.google.com/uc?export=download&id=1emLrrpFWT8dCoj5BJb12-5QMG2-nruUw [following]
--2022-05-09 08:28:09--  https://drive.google.com/uc?export=download&id=1emLrrpFWT8dCoj5BJb12-5QMG2-nruUw
Resolving drive.google.com (drive.google.com)... 74.125.195.138, 74.125.195.100, 74.125.195.113, ...
Connecting to drive.google.com (drive.google.com)|74.125.195.138|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://doc-10-10-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/air8m1ehsbshqbur1tmo66cpve2ijq90/1652084850000/17946651057176172524/*/1emLrrpFWT8dCoj5BJb12-5QMG2-nruUw?e=download [following]
--2022-05-09 08:28:10--  https://doc-10-10-docs.googleusercontent.com/docs/securesc/ha0ro937

In [9]:
# 라이브러리 불러오기

import pandas as pd
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from sklearn.ensemble import RandomForestClassifier

In [10]:
train = pd.read_csv('data/train.csv')

# Scaling
scaler = MinMaxScaler()
scaler.fit(train[['fixed acidity']])
train['Scaled fixed acidity'] = scaler.transform(train[['fixed acidity']])

# Encoding
encoder = OneHotEncoder()
encoder.fit(train[['type']])
onehot = encoder.transform(train[['type']])
onehot = onehot.toarray()
onehot = pd.DataFrame(onehot)
onehot.columns = encoder.get_feature_names()
train = pd.concat([train, onehot], axis = 1)
train = train.drop(columns = ['type'])
train.head()

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,index,quality,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,Scaled fixed acidity,x0_red,x0_white
0,0,5,5.6,0.695,0.06,6.8,0.042,9.0,84.0,0.99432,3.44,0.44,10.2,0.148760,0.0,1.0
1,1,5,8.8,0.610,0.14,2.4,0.067,10.0,42.0,0.99690,3.19,0.59,9.5,0.413223,1.0,0.0
2,2,5,7.9,0.210,0.39,2.0,0.057,21.0,138.0,0.99176,3.05,0.52,10.9,0.338843,0.0,1.0
3,3,6,7.0,0.210,0.31,6.0,0.046,29.0,108.0,0.99390,3.26,0.50,10.8,0.264463,0.0,1.0
4,4,6,7.8,0.400,0.26,9.5,0.059,32.0,178.0,0.99550,3.04,0.43,10.9,0.330579,0.0,1.0


2-a. 랜덤포레스트 분류 모형을 불러옵니다.

In [12]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score


2-b. 랜덤포레스트 분류 모형을 "random_classifier"라는 변수에 저장합니다.

In [11]:
random_classifier=RandomForestClassifier(n_estimators=20,max_depth=5, random_state=0)

2-c. "X"라는 변수에 train의 "quality" 피쳐를 제거하고 저장합니다.

In [14]:
X=train.drop(columns=['quality'])

2-d. "y"라는 변수에 정답인 train의 "quality" 피쳐를 저장합니다.

In [15]:
y=train['quality']

2-e. "random_classifier"를 X와 y를 이용해 학습시켜보세요.

In [17]:
random_classifier.fit(X,y)

RandomForestClassifier(max_depth=5, n_estimators=20, random_state=0)

# 3. 보팅

In [32]:
# 데이터 다운로드 링크로 데이터를 코랩에 불러옵니다.

!wget 'https://bit.ly/3i4n1QB'

import zipfile
with zipfile.ZipFile('3i4n1QB', 'r') as existing_zip:
    existing_zip.extractall('data')

--2022-05-09 08:44:13--  https://bit.ly/3i4n1QB
Resolving bit.ly (bit.ly)... 67.199.248.10, 67.199.248.11
Connecting to bit.ly (bit.ly)|67.199.248.10|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://drive.google.com/uc?export=download&id=1emLrrpFWT8dCoj5BJb12-5QMG2-nruUw [following]
--2022-05-09 08:44:13--  https://drive.google.com/uc?export=download&id=1emLrrpFWT8dCoj5BJb12-5QMG2-nruUw
Resolving drive.google.com (drive.google.com)... 74.125.142.139, 74.125.142.100, 74.125.142.102, ...
Connecting to drive.google.com (drive.google.com)|74.125.142.139|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://doc-10-10-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/ar0dagh7tdfnplrqfpelshpqlmacp0j5/1652085825000/17946651057176172524/*/1emLrrpFWT8dCoj5BJb12-5QMG2-nruUw?e=download [following]
--2022-05-09 08:44:14--  https://doc-10-10-docs.googleusercontent.com/docs/securesc/ha0ro937

In [33]:
# 라이브러리 및 데이터 불러오기
import pandas as pd
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier

from sklearn.ensemble import VotingClassifier
from sklearn.ensemble import RandomForestClassifier

train = pd.read_csv('data/train.csv')
test = pd.read_csv('data/test.csv')

train.drop('index',axis = 1 ,inplace =True)
test.drop('index',axis = 1 ,inplace =True)

In [35]:
# 데이터 확인
train.head(3)
test.head(3)

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,type
0,9.0,0.31,0.48,6.6,0.043,11.0,73.0,0.9938,2.90,0.38,11.6,white
1,13.3,0.43,0.58,1.9,0.070,15.0,40.0,1.0004,3.06,0.49,9.0,red
2,6.5,0.28,0.27,5.2,0.040,44.0,179.0,0.9948,3.19,0.69,9.4,white


In [36]:
# 원핫 인코딩 (pd.get_dummies())
train=pd.get_dummies(train,columns=['type'])
test=pd.get_dummies(test,columns=['type'])


In [37]:
# 모델 정의
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier

lr=LogisticRegression(solver='newton-cg')
knn=KNeighborsClassifier()

# VotingClassifier 정의
vote_clf = VotingClassifier(estimators=[('LR',lr), ('KNN', knn)], voting = 'soft')

In [39]:
# 모델 학습
# X 는 train에서 quality 를 제외한 모든 변수
# y 는 train의 qulity 변수
X=train.drop(columns=['quality'])
y=train['quality']

# fit 메소드를 이용해 모델 학습
vote_clf.fit(X,y)

VotingClassifier(estimators=[('LR', LogisticRegression(solver='newton-cg')),
                             ('KNN', KNeighborsClassifier())],
                 voting='soft')

In [44]:
test.head(3)

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,type_red,type_white
0,9.0,0.31,0.48,6.6,0.043,11.0,73.0,0.9938,2.90,0.38,11.6,0,1
1,13.3,0.43,0.58,1.9,0.070,15.0,40.0,1.0004,3.06,0.49,9.0,1,0
2,6.5,0.28,0.27,5.2,0.040,44.0,179.0,0.9948,3.19,0.69,9.4,0,1


In [47]:
# predict 메소드와 test_one 데이터를 이용해 품질 예측

pred = vote_clf.predict(test)
pred

array([6, 5, 5, 6, 6, 6, 6, 6, 6, 7, 5, 6, 7, 7, 5, 5, 6, 6, 6, 5, 6, 6,
       5, 6, 6, 5, 5, 5, 6, 5, 7, 5, 5, 5, 5, 6, 5, 5, 6, 5, 6, 5, 6, 6,
       6, 6, 5, 6, 5, 6, 5, 5, 6, 5, 5, 5, 5, 5, 6, 6, 5, 6, 5, 6, 6, 6,
       5, 6, 5, 5, 5, 5, 5, 5, 6, 6, 5, 6, 6, 6, 6, 6, 5, 5, 6, 5, 5, 5,
       7, 6, 6, 6, 5, 6, 6, 6, 6, 7, 6, 7, 5, 5, 5, 5, 5, 5, 6, 6, 6, 6,
       6, 6, 6, 6, 6, 6, 6, 7, 6, 6, 6, 5, 6, 7, 5, 5, 5, 6, 6, 6, 6, 5,
       6, 5, 5, 6, 6, 7, 5, 6, 6, 5, 6, 5, 5, 5, 6, 6, 6, 6, 6, 6, 6, 5,
       6, 5, 6, 5, 5, 5, 6, 6, 6, 6, 6, 6, 5, 6, 5, 5, 5, 5, 6, 5, 6, 6,
       6, 5, 5, 5, 6, 5, 5, 7, 5, 6, 5, 6, 5, 6, 6, 5, 6, 6, 6, 6, 7, 6,
       5, 5, 5, 5, 5, 5, 5, 5, 6, 7, 6, 6, 6, 5, 5, 6, 6, 6, 5, 6, 6, 6,
       6, 6, 6, 6, 7, 5, 6, 6, 6, 5, 6, 7, 7, 5, 6, 6, 5, 5, 6, 6, 6, 6,
       6, 6, 6, 6, 5, 6, 6, 7, 5, 6, 6, 7, 6, 6, 6, 7, 7, 5, 6, 5, 6, 6,
       6, 7, 6, 5, 6, 6, 6, 6, 6, 5, 6, 5, 6, 7, 5, 6, 6, 7, 5, 5, 7, 6,
       6, 7, 5, 6, 7, 5, 5, 7, 5, 6, 8, 6, 6, 5, 6,